In [ ]:
import numpy as np
import gudhi
import matplotlib.pyplot as plt
import matplotlib as mpl

import scipy.spatial.distance as dist
import itertools

import iblofunmatch.inter as ibfm
import iblofunmatch.topological_data_quality_0 as tdq0
output_dir = "output" 
plots_dir = "plots/example_computation/"

import os 
os.makedirs(output_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

# Computation of Block Function in dimension 0

Consider the following example, with points taken from a sample.

We consider 7 points and a sample of three points. 

In [ ]:
RandGen = np.random.default_rng(2)
# # Generate Random Sample
X = ibfm.sampled_circle(0,2,6, RandGen)
print(X)
S_indices = RandGen.choice(X.shape[0],3, replace=False)
# Sort and save 
S_compl = np.ones(X.shape[0], dtype="bool")
S_compl[S_indices] = False
X = np.vstack((X[S_indices], X[S_compl]))
X[3] = [-0.1,0]
X[4] = [0.4,0]
X[5] = [0.15,np.sqrt(0.5**2 - 0.25**2)]
S_indices = range(len(S_indices))
np.savetxt("Z_example_new.txt", X, fmt="%.4f")
np.savetxt("X_idx_example_new.txt", S_indices, fmt="%d")
# Load X and S_indices from files 
X = np.loadtxt("Z_example_new.txt")
S_indices = np.loadtxt("X_idx_example_new.txt", dtype="int")
S = X[S_indices]
# Plot point cloud
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0.png")

We plot, for illustration, the Vietoris-Rips complex at a sequence of values

In [ ]:
def plot_Vietoris_Rips_subset(X, S_indices, filt_val, ax, labels=False, fontsize=15):
    S = X[S_indices]
    # Plot point cloud
    if labels:
        ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=230, marker="o", zorder=2)
        ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=230, marker="o", zorder=1)
    else:
        ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
        ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="o", zorder=1)
    # Plot simplicial complex 
    rips_complex = gudhi.RipsComplex(points=X, max_edge_length=filt_val)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=1)
    simplex_tree.expansion(2)
    edgelist = []
    for filtered_value in simplex_tree.get_filtration():
        simplex = filtered_value[0]
        if len(simplex)==2:
            edgelist.append(simplex)
            if len(np.intersect1d(simplex, S_indices))==2:
                ax.plot(X[simplex][:,0], X[simplex][:,1], linewidth=2, c=mpl.colormaps["RdBu"](0.3/1.3), zorder=0.5)
            else:
                ax.plot(X[simplex][:,0], X[simplex][:,1], linewidth=2, c=mpl.colormaps["RdBu"](1/1.3), zorder=0.5)
        # if len(simplex)==3:
        #     ax.add_patch(mpl.patches.Polygon(X[simplex], closed=True, facecolor="black", alpha=0.3, zorder=0.2))
    ax.set_aspect("equal")
    # Adjust margins
    xscale = ax.get_xlim()[1]-ax.get_xlim()[0]
    yscale = ax.get_ylim()[1]-ax.get_ylim()[0]
    xlim = ax.get_xlim()
    xlim = (xlim[0]-xscale*0.1, xlim[1]+xscale*0.1)
    ax.set_xlim(xlim)
    ylim = ax.get_ylim()
    ylim = (ylim[0]-yscale*0.1, ylim[1]+yscale*0.1)
    ax.set_ylim(ylim)
    # Plot labels
    if labels:
        components = ibfm.compute_components(edgelist, X.shape[0])
        # Point Labels 
        for i in range(X.shape[0]):
            ax.text(X[i,0]-0.035*xscale, X[i,1]-0.035*yscale, f"{components[i]}", fontsize=fontsize, color="white", fontweight="bold")
    # Finish with aspect details 
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
filtrations = [0.0, 1, 2, 3]
fig, ax = plt.subplots(ncols=len(filtrations), figsize=(3*len(filtrations),3))
for j, filt_val in enumerate(filtrations):
    plot_Vietoris_Rips_subset(X, S_indices, filt_val, ax[j])
    # Set title 
    ax[j].set_title(f"{filt_val:1.1f}") 
    #ax[j].set_title(f"VR_{filt_val:1.1f}(X) \subseteq VR_{filt_val:1.1f}(Z)") 
plt.savefig("VR_filtration.png")


Repeat computation of Vietoris-Rips complex with labels on vertices and components.

In [ ]:
filtrations = [0,1,1.1,1.2, 2.5]
fig, ax = plt.subplots(nrows=2, ncols=len(filtrations), figsize=(2.5*len(filtrations),6))
for i, filt_val in enumerate(filtrations):
    plot_Vietoris_Rips_subset(X, S_indices, filt_val, ax[0, i], labels=True)
    S = X[S_indices]
    plot_Vietoris_Rips_subset(S, [], filt_val, ax[1, i], labels=True)
    # Plot point cloud extra large
    ax[0,i].set_title(f"{filt_val:.1f}")

ax[0,0].set_title("Dataset")
ax[1,0].set_title("Subset")
plt.tight_layout()
plt.savefig("VR_components.png")

Next, we compute the block function induced by the inclusion $S\hookrightarrow X$

In [ ]:
ibfm_out = ibfm.get_IBloFunMatch_subset(None, X, S_indices, output_dir, num_it=4, max_rad=-1, points=True, store_0_pm=True)

In [ ]:
ibfm_out["S_barcode_0"]

In [ ]:
ibfm_out["X_barcode_0"]

Now, we compare result with computation using Kruskal.

In [ ]:
# Compute matching using minimum spanning trees
filtration_list_X, pairs_arr_X = tdq0.filtration_pairs(X)
filtration_list_S, pairs_arr_S = tdq0.filtration_pairs(S)
F = tdq0.get_inclusion_matrix(pairs_arr_S, pairs_arr_X, S_indices)
matching = tdq0.get_inclusion_matrix_pivots(F, X.shape[0])
# Check that both outputs coincide
assert(np.all((ibfm_out["X_barcode_0"][:,1] - filtration_list_X)<1e-5))
assert(np.all((ibfm_out["S_barcode_0"][:,1] - filtration_list_S)<1e-5))
assert(np.all((np.array(ibfm_out["induced_matching_0"],dtype="int") - np.array(matching, dtype="int")==0)))

Now, we describe the $0$-persistence barcodes in terms of evolution of components.

In [ ]:
def plot_merge_tree(barcode_0, reps_0, ax):
    max_x = np.max(barcode_0)*1.1
    num_points = barcode_0.shape[0]+1
    y= np.linspace(0, 0.3*num_points, num_points)
    idx_death = []
    merging_into= []
    death_val = []
    for idx, (bar, rep) in enumerate(zip(barcode_0, reps_0)):
        ax.plot(bar, [y[idx], y[idx]], c=mpl.colormaps["RdBu"](1/1.3), linewidth=3, zorder=0.5)
        idx_death.append(np.max(rep))
        merging_into.append(np.min(rep))
        death_val.append(bar[1])
    
    # merge lines in red
    idx_death.append(0)
    for idx, (j, death) in enumerate(zip( merging_into, death_val)):
        death_merging = idx_death.index(j)
        ax.plot([death, death], [y[idx],y[death_merging]], linewidth=3, c=mpl.colormaps["RdBu"](0.3/1.3), zorder=0.5)

    xscale = ax.get_xlim()[1]-ax.get_xlim()[0]
    yscale = ax.get_ylim()[1]-ax.get_ylim()[0]
    for i, idx in enumerate(idx_death):
        ax.text(-0.015*xscale, y[i]-0.04*yscale, f"{idx}", zorder=0.7, fontsize=10, color="white", fontweight="bold")
        if i < len(idx_death)-1:
            death_x = barcode_0[i,1]
            ax.text(death_x-0.015*xscale, y[i]-0.04*yscale, f"{merging_into[i]}", zorder=0.7, fontsize=10, color="white", fontweight="bold")

    ax.scatter(np.zeros(len(y)),y, s=100, marker="o", color=mpl.colormaps["RdBu"](1/1.3), zorder=0.6)
    ax.scatter(barcode_0[:,1], y[:-1], s=100, marker="o", color=mpl.colormaps["RdBu"](0.3/1.3), zorder=0.6)
    ax.set_xlim(ax.get_xlim()[0]-0.1*xscale, ax.get_xlim()[1]+0.1*xscale)
    ax.set_ylim(ax.get_ylim()[0]-0.1*yscale, ax.get_ylim()[1]+0.1*yscale)
    # Top horizontal interval
    ax.plot([0,max_x*2], [y[-1],y[-1]], linewidth=3, c=mpl.colormaps["RdBu"](1/1.3), zorder=0.5)
    ax.set_yticks([])

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(9,2.4))
plot_merge_tree(ibfm_out["S_barcode_0"], ibfm_out["S_reps_0"], ax[0])
plot_merge_tree(ibfm_out["X_barcode_0"], ibfm_out["X_reps_0"], ax[1])
ax[0].set_title("Merge tree from X")
ax[1].set_title("Merge tree from Z")
ylim = ax[1].get_ylim()
ax[1].plot([0.8,0.8], ylim, linewidth=1, color="gray", linestyle="--", zorder=0)
plt.tight_layout()
plt.savefig("merge_trees_X_Z.png")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_matching(ibfm_out, ax, fig, block_function=True, dim=0)
plt.savefig(plots_dir + "block_function_0.png")

New plot representation

In [ ]:
from importlib import reload
reload(ibfm)

In [ ]:
fig, ax = plt.subplots(figsize=(7,2.5))
ibfm.plot_matching_0(ibfm_out, ax)
plt.tight_layout()
plt.savefig(plots_dir + "block_matching_0.png")

## Geometric intuition for previous computation

Now, we perform the previous computation in terms of connected components. First, we plot again the points with numbers besides them.

In [ ]:
S = X[S_indices]
fig, ax = plt.subplots(ncols=1, figsize=(3,3))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
for idx in range(X.shape[0]):
    ax.text(X[idx,0]+0.05, X[idx,1]+0.05, f"{idx}", fontsize=10)
ax.set_axis_off()
plt.savefig(plots_dir + "points_0_numbered.png")

Denote by $a_1, a_2$ the endpoints from $\textrm{PH}_0(X)$ such that $a_1 < a_2$. Also, denote $b_1 < \cdots < b_5$ the endpoints from the barcode of $\textrm{PH}_0(Z)$.

Now, we show that $\mathcal{M}_f(a_1, b_5) = \textrm{dim}(\textrm{H}_0(G(X,Z)_{a_1,b_5}) = 1$

In [ ]:
S = X[S_indices]
fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(12,9))
for idx, pair_ab in enumerate([[0,3],[1,2], [0,4], [1,3]]):
    a = ibfm_out["S_barcode_0"][:,1][pair_ab[0]]
    b = ibfm_out["X_barcode_0"][:,1][pair_ab[1]]
    ibfm.plot_geometric_matching(a, b, S_indices, X, ibfm_out, ax[idx])
plt.tight_layout()
plt.savefig(plots_dir + "matching_geometric_6pts.png")

# Additional example

Let us compute an additional example to check that the intuition about matchings and cycles holds.

In [ ]:
X = np.array([[0, 0],[2,0], [4,0], [1,0.3],[3,-0.3]])
S_indices = list(range(3))
S = X[S_indices]
ibfm_out = ibfm.get_IBloFunMatch_subset(None, X, S_indices, output_dir, num_it=4, max_rad=-1, points=True, store_0_pm=True)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(12,3))
ax = [ax]
for idx, pair_ab in enumerate([[0,0]]):
    a = ibfm_out["S_barcode_0"][:,1][pair_ab[0]]
    b = ibfm_out["X_barcode_0"][:,1][pair_ab[1]]
    print(f"a:{a}, b:{b}")
    ibfm.plot_geometric_matching(a, b, S_indices, X, ibfm_out, ax[idx], _tol=1e-3)

plt.tight_layout()
plt.savefig(plots_dir + "matching_geometric_aligned.png")

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(3,0.8))
ax.scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=60, marker="o", zorder=2)
ax.scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=40, marker="x", zorder=1)
for idx in range(X.shape[0]):
    ax.text(X[idx,0]+0.05, X[idx,1]+0.05, f"{idx}", fontsize=10)

ax.set_aspect("equal")
ylim = ax.get_ylim()
yrange = ylim[1]-ylim[0]
ylim = (ylim[0] - yrange*0.1, ylim[1] + yrange*0.1)
ax.set_ylim(ylim)
plt.tight_layout()
plt.savefig(plots_dir + "points_aligned.png")

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_matching(ibfm_out, ax, fig, block_function=True, dim=0)
plt.savefig(plots_dir + "block_function_0_aligned.png")

In [ ]:
fig, ax = plt.subplots(figsize=(3,3))
ax.set_ylim([-1.5,1.5])
plot_Vietoris_Rips_subset(X, S_indices, 0, ax, labels=True, fontsize=10)

In [ ]:
ibfm_out["X_barcode_0"]

# Computation of Block Function in dimension 1

Consider the following example, with points taken from a few circles.

In [ ]:
RandGen = np.random.default_rng(2)
C0 = ibfm.sampled_circle(2.3,2.5,40, RandGen)
C1 = ibfm.sampled_circle(1,1.1,40, RandGen)-[1.1,0]
C2 = ibfm.sampled_circle(1,1.1,40, RandGen)+[1.1,0]
X = np.vstack([C0, C1, C2])
S_indices = list(range(C0.shape[0]))
S_indices += list(np.nonzero(C1[:,1]<0)[0]+C0.shape[0])
S_indices += list(np.nonzero(C2[:,1]>0)[0]+C0.shape[0]+ C1.shape[0])
S = X[S_indices]
fig, ax = plt.subplots(ncols=2, figsize=(6,3))
ax[0].scatter(S[:,0], S[:,1], color=mpl.colormaps["RdBu"](0.3/1.3), s=50, marker="o", zorder=2)
ax[1].scatter(X[:,0], X[:,1], color=mpl.colormaps["RdBu"](1/1.3), s=50, marker="x", zorder=1)
ax[0].set_axis_off()
ax[1].set_axis_off()
plt.savefig(plots_dir + "points_1.png")

In [ ]:
ibfm_out = ibfm.get_IBloFunMatch_subset(None, X, S_indices, output_dir, num_it=4, max_rad=-1, points=True, store_0_pm=True)

In [ ]:
min_length=0.3
S_barcode = ibfm_out["S_barcode_1"]
S_long = np.nonzero(S_barcode[:,1]-S_barcode[:,0] > min_length)[0].tolist()
X_barcode = ibfm_out["X_barcode_1"]
X_long = np.nonzero(X_barcode[:,1]-X_barcode[:,0] > min_length)[0].tolist()

In [ ]:
X_long

In [ ]:
S_long

In [ ]:
X_barcode_long = X_barcode[X_long]
S_barcode_long = S_barcode[S_long]

In [ ]:
ibfm_out["block_function_1"]

In [ ]:
blofun_1_long = [X_long.index(row) for col, row in enumerate(ibfm_out["block_function_1"]) if col in S_long]

In [ ]:
pm_matrix_1_long = [[X_long.index(row_idx) for row_idx in column if row_idx in X_long]  for col_idx, column in enumerate(ibfm_out["pm_matrix_1"]) if col_idx in S_long]

In [ ]:
pm_matrix_1_long 

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(5,2))
ibfm.plot_from_block_function(S_barcode_long, X_barcode_long, blofun_1_long, fig, ax)
plt.savefig(plots_dir + "block_function_1.png")